In [1]:
from pyspark.sql import SparkSession, functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType

In [11]:
spark = SparkSession.builder.master('local[*]').appName('HTF_Vol').getOrCreate()
spark.conf.set("spark.executor.memory", "2g")

df = spark.read.csv('newaa.csv', header=True)
df = df.withColumn('TIME', f.date_format(df.TIME, 'HH:mm:ss'))
df = df.withColumn('PRICE', df['PRICE'].cast(DoubleType()))
df = df.withColumn('SIZE', df['SIZE'].cast(DoubleType()))

In [12]:
# patition a window to calculcate the u_sequence
w = Window().partitionBy('SYMBOL', 'DATE').orderBy('TIME')

# calculate the u_sequence
df = df.withColumn('PRICE_1', f.lead('PRICE').over(w))
df = df.withColumn('LOG_RETURN', f.log(df.PRICE_1/df.PRICE))


# patition a window to calculate the moving standard deviation
window_period = 100
w2 = Window().partitionBy('SYMBOL', 'DATE').orderBy('TIME').rowsBetween(0, window_period)
df = df.withColumn('VOLATILITY', f.stddev(df['LOG_RETURN']).over(w2))

In [6]:
df.show()

+------+--------+--------+-----+------+-------+--------------------+--------------------+
|SYMBOL|    DATE|    TIME|PRICE|  SIZE|PRICE_1|          LOG_RETURN|          VOLATILITY|
+------+--------+--------+-----+------+-------+--------------------+--------------------+
|   AAI|20021211|09:30:43| 3.77|1400.0|   3.77|                 0.0|0.006507521208565777|
|   AAI|20021211|09:30:44| 3.77| 100.0|   3.77|                 0.0|0.006510356653572...|
|   AAI|20021211|09:30:44| 3.77| 100.0|   3.77|                 0.0|0.006510356653572328|
|   AAI|20021211|09:30:59| 3.77| 200.0|   3.77|                 0.0|0.006510356653572...|
|   AAI|20021211|09:30:59| 3.77| 800.0|   3.81|0.010554187678690171|0.006510356653572...|
|   AAI|20021211|09:33:11| 3.81|1000.0|    3.8|-0.00262812240626...|0.006432192089653349|
|   AAI|20021211|09:33:25|  3.8|1100.0|    3.8|                 0.0|0.006474316635355122|
|   AAI|20021211|09:35:08|  3.8| 100.0|    3.8|                 0.0|0.006474316635355123|
|   AAI|20

In [13]:
seconds = 300
timeFormat = 'HH:mm:ss'
seconds_window = f.from_unixtime(f.unix_timestamp('TIME', format='HH:mm:ss') - 
                  f.unix_timestamp('TIME', format='HH:mm:ss') % seconds, format='HH:mm:ss')

In [14]:
df = df.withColumn('5_min_window', seconds_window)

#df_AA = df.where(df['SYMBOL'] == 'A').select('*')

In [15]:
new_df = df.groupby('SYMBOL', 'DATE', '5_min_window').agg(
    f.avg('VOLATILITY').alias('avgVol'),
    f.avg('PRICE').alias('avgPrice'),
    f.avg('SIZE').alias('avgSize'))

In [16]:
new_df.show()

+------+--------+------------+--------------------+------------------+------------------+
|SYMBOL|    DATE|5_min_window|              avgVol|          avgPrice|           avgSize|
+------+--------+------------+--------------------+------------------+------------------+
|   AAI|20021211|    09:30:00|0.006493636649694794|3.7800000000000002| 671.4285714285714|
|   AAI|20021211|    09:35:00|0.006503985745986441|             3.804|             220.0|
|   AAI|20021211|    09:40:00|0.006428111944606004|              3.84|             100.0|
|   AAI|20021211|    09:45:00| 0.00654065911641249|3.8737500000000002|            1125.0|
|   AAI|20021211|    09:50:00|0.006612866177576418|3.9170000000000003|             380.0|
|   AAI|20021211|    09:55:00|0.006335501674427891|3.8850000000000002|             475.0|
|   AAI|20021211|    10:00:00|0.005581250230737032|3.8833333333333333|             450.0|
|   AAI|20021211|    10:05:00|0.004583329886594783|3.8800000000000003|            3200.0|
|   AAI|20

In [17]:
w3 = Window().partitionBy('SYMBOL', 'DATE').orderBy('5_min_window')

# calculate the u_sequence
new_df = new_df.withColumn('PRICE_1', f.lead('avgPrice').over(w3))
new_df = new_df.withColumn('U_SEQ', f.log(new_df.PRICE_1/new_df.avgPrice))
new_df = new_df.withColumn('Num', f.lit(1))
new_df = new_df.withColumn('totalSum', f.sum('Num').over(w3))
new_df = new_df.withColumn('sumSqaures', f.sum(f.pow(new_df.U_SEQ, 2)).over(w3)/(new_df.totalSum - f.lit(1)))
new_df = new_df.withColumn('suqareSums', f.pow(f.sum(new_df.U_SEQ).over(w3), 2)/((new_df.totalSum - f.lit(1)) 
                                                                                * new_df.totalSum))
new_df = new_df.withColumn('sectionVol', f.sqrt(new_df.sumSqaures - new_df.suqareSums))




#section_volatility = f.sqrt(f.sum(f.pow(new_df.U_SEQ, 2)).over(w3)/(new_df.totalSum - f.lit(1)) 
#                            - f.pow(f.sum(new_df.U_SEQ).over(w3), 2)/((new_df.totalSum - f.lit(1) * new_df.totalSum)))


#new_df = new_df.withColumn('sectionVol', section_volatility)
#df = df.withColumn('cumSum', f.sum('PRICE').over(w))

#df = df.withColumn('count', f.lit(1))
#df = df.withColumn('totalNum', f.sum('count').over(w))

In [62]:
new_df.show()

+------+--------+------------+--------------------+------------------+------------------+------------------+--------------------+---+--------+--------------------+--------------------+--------------------+
|SYMBOL|    DATE|5_min_window|              avgVol|          avgPrice|           avgSize|           PRICE_1|               U_SEQ|Num|totalSum|          sumSqaures|          suqareSums|          sectionVol|
+------+--------+------------+--------------------+------------------+------------------+------------------+--------------------+---+--------+--------------------+--------------------+--------------------+
|   AAI|20021211|    09:30:00|0.006493636649694794|3.7800000000000002| 671.4285714285714|             3.804| 0.00632913505164753|  1|       1|                null|                null|                null|
|   AAI|20021211|    09:35:00|0.006503985745986441|             3.804|             220.0|              3.84|0.009419221916491558|  1|       2|1.287796920141082...|1.24005373597

In [22]:
data_pd = new_df\
    .select('SYMBOL', 'DATE', '5_min_window', 'avgVol', 'avgPrice', 'avgSize', 'sectionVol')\
    .toPandas()

In [23]:
data_pd.head()

,SYMBOL,DATE,5_min_window,avgVol,avgPrice,avgSize,sectionVol
0,AAI,20021211,09:30:00,0.006494,3.78000,671.428571,NaN
1,AAI,20021211,09:35:00,0.006504,3.80400,220.000000,0.002185
2,AAI,20021211,09:40:00,0.006428,3.84000,100.000000,0.001626
3,AAI,20021211,09:45:00,0.006541,3.87375,1125.000000,0.001979
4,AAI,20021211,09:50:00,0.006613,3.91700,380.000000,0.007839


In [13]:
df_pd = data_pd.copy()

In [14]:
df_pd.columns = ['Symbol', 'Date', 'Time', 'Vol', 'Price', 'Size', 'Price_1', 'U_seq']

In [15]:
df_pd.head()

,Symbol,Date,Time,Vol,Price,Size,Price_1,U_seq
0,AAI,20021211,09:30:00,0.006494,3.78000,671.428571,3.80400,0.006329
1,AAI,20021211,09:35:00,0.006504,3.80400,220.000000,3.84000,0.009419
2,AAI,20021211,09:40:00,0.006428,3.84000,100.000000,3.87375,0.008751
3,AAI,20021211,09:45:00,0.006541,3.87375,1125.000000,3.91700,0.011103
4,AAI,20021211,09:50:00,0.006613,3.91700,380.000000,3.88500,-0.008203


In [16]:
import numpy as np
def sectionVol(df):
    data = df.copy()
    
    for i in data.index:
        if i == 0:
            data.loc[i, 'Section_Vol'] = np.nan
        else:
            data.loc[i, 'Section_Vol'] = np.sqrt(np.sum(np.power(data.loc[:i, 'U_seq'], 2))/i 
                                                 - np.power(np.sum(data.loc[:i, 'U_seq']), 2)/(i*(i+1)))
    
    print(data.head())
    return data

In [17]:
sectionVolatility = {}
for symbol in set(df_pd.Symbol):
    sectionVolatility[symbol] = sectionVol(df_pd.loc[df_pd.Symbol == symbol, :])

    Symbol      Date      Time       Vol      Price         Size    Price_1  \
299     AA  20021219  09:30:00  0.000962  22.953175  1980.952381  23.043529   
300     AA  20021219  09:35:00  0.000870  23.043529  2547.058824  23.080526   
301     AA  20021219  09:40:00  0.001052  23.080526  2142.105263  23.158947   
302     AA  20021219  09:45:00  0.001102  23.158947  1721.052632  23.237826   
303     AA  20021219  09:50:00  0.001102  23.237826   943.478261  23.255652   

        U_seq  Section_Vol  
299  0.003929     0.000227  
300  0.001604     0.000244  
301  0.003392     0.000312  
302  0.003400     0.000367  
303  0.000767     0.000368  


  Symbol      Date      Time       Vol    Price         Size  Price_1  \
0    AAI  20021211  09:30:00  0.006494  3.78000   671.428571  3.80400   
1    AAI  20021211  09:35:00  0.006504  3.80400   220.000000  3.84000   
2    AAI  20021211  09:40:00  0.006428  3.84000   100.000000  3.87375   
3    AAI  20021211  09:45:00  0.006541  3.87375  1125.000000  3.91700   
4    AAI  20021211  09:50:00  0.006613  3.91700   380.000000  3.88500   

      U_seq  Section_Vol  
0  0.006329          NaN  
1  0.009419     0.002185  
2  0.008751     0.001626  
3  0.011103     0.001979  
4 -0.008203     0.007839  


    Symbol      Date      Time       Vol      Price         Size    Price_1  \
221      A  20030313  09:30:00  0.001187  12.875778  1637.777778  12.829583   
222      A  20030313  09:35:00  0.000867  12.829583  1300.000000  12.809118   
223      A  20030313  09:40:00  0.000849  12.809118   923.529412  12.752963   
224      A  20030313  09:45:00  0.000752  12.752963   733.333333  12.721842   
225      A  20030313  09:50:00  0.000762  12.721842   836.842105  12.690357   

        U_seq  Section_Vol  
221 -0.003594     0.000241  
222 -0.001596     0.000263  
223 -0.004394     0.000393  
224 -0.002443     0.000423  
225 -0.002478     0.000452  


    Symbol      Date      Time       Vol      Price         Size    Price_1  \
377    AAP  20030313  09:30:00  0.000664  40.858750   625.000000  41.028846   
378    AAP  20030313  09:35:00  0.000499  41.028846  1273.076923  41.003571   
379    AAP  20030313  09:40:00  0.000505  41.003571   553.571429  40.860556   
380    AAP  20030313  09:45:00  0.000450  40.860556   422.222222  40.887407   
381    AAP  20030313  09:50:00  0.000454  40.887407   618.518519  40.821053   

        U_seq  Section_Vol  
377  0.004154     0.000214  
378 -0.000616     0.000216  
379 -0.003494     0.000281  
380  0.000657     0.000282  
381 -0.001624     0.000294  


In [23]:
%matplotlib
from matplotlib import pyplot as plt
import pandas as pd
fig, axes = plt.subplots(nrows=2, ncols=1)

for symbol in sectionVolatility:
    plt.subplot(2, 1, 1)
    plt.plot(sectionVolatility[symbol].loc[:,  'Section_Vol'], label=symbol)
    plt.legend(loc='upper right')
    plt.subplot(2, 1, 2)
    plt.plot(sectionVolatility[symbol].loc[:,  'Price'], label=symbol)
    sectionVolatility[symbol].to_csv(symbol + '_SectionVol.csv')

plt.suptitle('Section Volatility')
#plt.ylabel('Section Volatility')
plt.savefig('Section_Vol.png')

Using matplotlib backend: MacOSX


In [123]:
new_df.loc[:, 'd'] = pd.to_datetime(new_df.loc[:, 'Date'] + ' ' + new_df.loc[:, 'Time'])
new_df.dtypes

Date                   object
Time                   object
Vol                   float64
Price                 float64
Size                  float64
Price_1               float64
U_seq                 float64
Section_Vol           float64
d              datetime64[ns]
dtype: object